## Creating dataset

In [1]:
import os
os.chdir('../mimic')
from process_mimic import create_merged, get_first_data
import pandas as pd
from scipy import stats
import errno

Run process_mimic functions to get daily and admission data.

In [2]:
try:
    os.makedirs('data_processed')
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

In [3]:
%%time
df = create_merged()
df_first = get_first_data(df)

[    INFO |     process_mimic.py:337]   Starting preprocessing of MIMIC data
[   DEBUG |     process_mimic.py: 70]   Reading chartevents file data_raw\CHARTEVENTS.csv.gz
[   DEBUG |     process_mimic.py: 74]   Read chartevents file data_raw\CHARTEVENTS.csv.gz
[   DEBUG |     process_mimic.py: 90]   Running process for chart event tobacco over 0 rows
[   DEBUG |     process_mimic.py: 90]   Running process for chart event CRP over 743 rows
[   DEBUG |     process_mimic.py: 90]   Running process for chart event Haemoglobin over 137117 rows
[   DEBUG |     process_mimic.py: 90]   Running process for chart event AST over 203 rows
[   DEBUG |     process_mimic.py: 90]   Running process for chart event Diabetes over 0 rows
[   DEBUG |     process_mimic.py: 90]   Running process for chart event PaCO2 over 140215 rows
[   DEBUG |     process_mimic.py: 90]   Running process for chart event apache2 over 19 rows
[   DEBUG |     process_mimic.py: 90]   Running process for chart event sofa over 3 ro

[   DEBUG |     process_mimic.py:101]   Saving chart events to data_processed/mimic_chartevents.csv
[   DEBUG |     process_mimic.py:115]   Creating day blocks
[   DEBUG |     process_mimic.py:124]   Created day blocks. 218696 rows, 38385 unique admissions
[   DEBUG |     process_mimic.py:140]   Adding derived columns
[   DEBUG |        operations.py:  6]   Adding BMI
[    INFO |             utils.py:129]   Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
[    INFO |             utils.py:141]   NumExpr defaulting to 8 threads.
[   DEBUG |        operations.py: 13]   Adding PaO2/FiO2
[    INFO |     process_mimic.py:261]   Getting admission data
[    INFO |     process_mimic.py:190]   Getting procedures
[    INFO |     process_mimic.py:224]   Getting all diagnoses
[   DEBUG |     process_mimic.py:255]   Found 4724 Chronic Kidney Disease patients
[   DEBUG |     process_mimic.py:255]   Found 2670 Obesity patients
[   DEBUG |     process_mim

Wall time: 8min 35s


In [4]:
# make these columns (normally in first day only) present accross all days
df = df.drop(['Weight_kg', 'Height_inches', 'BMI'], axis=1).merge(
    df_first[['Weight_kg', 'Height_inches', 'BMI']], on='HADM_ID')

In [32]:
df.count().sort_values(ascending=False)

index                             218696
Obesity                           218696
SUBJECT_ID                        218696
ADMITTIME                         218696
DAYS_TO_DISCH                     218696
DEATH                             218696
GENDER                            218696
AGE                               218696
Asthma                            218696
Chronic Kidney Disease            218696
Heart disease                     218696
CHARTDATE                         218696
HADM_ID                           218696
Day                               218696
DIAGNOSIS                         218695
HR                                205363
RR                                204675
SaO2                              200038
Weight_kg                         196484
Potassium                         148614
Creatinine                        148460
Sodium                            147400
Height_inches                     144717
BMI                               126938
FiO2            

In [33]:
df_first.count().sort_values(ascending=False)

Day                               38385
Asthma                            38385
CHARTDATE                         38385
SUBJECT_ID                        38385
ADMITTIME                         38385
DAYS_TO_DISCH                     38385
DEATH                             38385
GENDER                            38385
AGE                               38385
index                             38385
Chronic Kidney Disease            38385
Obesity                           38385
Heart disease                     38385
DIAGNOSIS                         38384
HR                                35951
RR                                35924
SaO2                              35387
Weight_kg                         31826
Creatinine                        31633
Potassium                         31629
Sodium                            31618
HCO3                              24061
Platelet_count                    23984
Haemoglobin                       23983
arterial_bp_systolic              21919


### Invasive ventilation
The majority of the patients are ventilated at day 0. This could provide a problem for machine learning. (overfitting)


Should probably split problems into "requires ventilation" and "days to ventilation

Hard to do "requires ventilation" as most of them are on day 0, meaning can't really use first day data?

In [7]:
df_first['invasive_ventilation_DAYSTO'].count()

8860

In [8]:
df_first['invasive_ventilation_DAYSTO'].value_counts()

 0.0      4695
 1.0      1631
 2.0       627
 3.0       437
 4.0       331
 5.0       219
 6.0       212
 7.0       146
 9.0        82
 8.0        71
 10.0       65
 11.0       51
 13.0       38
 12.0       38
 14.0       25
 15.0       23
 16.0       20
 18.0       20
 19.0       17
 21.0       12
 20.0       11
 23.0        9
 17.0        8
 25.0        6
 22.0        5
 29.0        5
 35.0        5
 32.0        5
 26.0        5
 28.0        5
 27.0        4
 24.0        3
 31.0        3
 34.0        2
 37.0        2
 33.0        2
 56.0        1
 43.0        1
-1.0         1
 49.0        1
 64.0        1
 76.0        1
 112.0       1
 90.0        1
 44.0        1
 38.0        1
 48.0        1
 30.0        1
 40.0        1
 51.0        1
 46.0        1
 36.0        1
 170.0       1
 54.0        1
 82.0        1
 55.0        1
Name: invasive_ventilation_DAYSTO, dtype: int64

### Discharge
Every patient with an admission also has a discharge

In [9]:
df_first['DAYS_TO_DISCH'].count()

38385

In [10]:
df_first['DAYS_TO_DISCH'].value_counts()

 4      3931
 2      3417
 3      3397
 5      3233
 6      2837
 7      2534
 1      2187
 8      2031
 9      1736
 10     1407
 0      1319
 11     1128
 12      958
 13      898
 14      728
 15      659
 16      543
 17      486
 18      426
 20      381
 19      348
 21      311
 22      285
 23      249
 24      220
 28      181
 27      178
 26      173
 25      172
 29      145
 30      112
 35      111
 31      110
 33       97
 34       89
 32       78
 36       71
 37       65
 38       58
 41       57
 39       55
-1        52
 42       51
 40       50
 45       43
 43       42
 46       39
 44       35
 51       35
 48       31
 56       31
 52       27
 47       27
 50       24
 49       23
 55       23
 53       22
 54       21
 66       17
 57       17
 59       16
 58       16
 61       15
 64       15
 75       14
 60       13
 62       13
 68       12
 76       11
 72       11
 69       11
 71        9
 63        9
 78        9
 97        8
 65        8
 70        7

### Death
Around 10% mortality rate. Lots after day 0.

In [11]:
df_first['DAYS_TO_DEATH'].count()

3957

In [12]:
df_first['DAYS_TO_DEATH'].value_counts()

 0.0      600
 1.0      385
 2.0      303
 3.0      250
 4.0      230
 5.0      216
 6.0      206
 7.0      166
 8.0      155
 9.0      135
 10.0     112
 11.0     100
 12.0      87
 15.0      85
 13.0      80
 14.0      68
 16.0      67
 17.0      50
 18.0      45
-1.0       45
 21.0      35
 23.0      35
 19.0      34
 20.0      33
 22.0      28
 24.0      26
 26.0      26
 27.0      25
 25.0      23
 29.0      21
 35.0      19
 33.0      19
 28.0      19
 31.0      13
 34.0      13
 41.0      13
 37.0      12
 32.0      11
 30.0      11
 36.0      10
 51.0      10
 38.0       9
 45.0       9
 42.0       8
 40.0       7
 56.0       6
 46.0       5
 64.0       5
 44.0       5
 39.0       4
 43.0       3
 57.0       3
 50.0       3
 62.0       3
 69.0       3
 54.0       3
 52.0       3
 65.0       2
 100.0      2
 66.0       2
 87.0       2
 47.0       2
 55.0       2
 53.0       2
 106.0      2
 68.0       2
 49.0       2
 48.0       2
 72.0       2
 60.0       2
 102.0      1
 73.0 

### Choose features
Must choose a subset of features, as some of these have almost no data.
Choose features with most rows

TODO add ventilatory related features

In [34]:
numeric = [
    'Creatinine', 'HCO3', 'HR', 'Haemoglobin',
    'Platelet_count', 'Potassium', 'RR',
    'Sodium', 'Weight_kg', 'SaO2',
    'arterial_bp_systolic', 'arterial_bp_diastolic'
]

my_cols = numeric + [
    'HADM_ID', 'CHARTDATE', 
    'DAYS_TO_DISCH', 'AGE', 'GENDER', 'Day', 'DEATH',
    "Chronic Kidney Disease", "Obesity", "Heart disease", "Asthma", # comorbidities
]

In [35]:
df.count()

index                             218696
HADM_ID                           218696
CHARTDATE                         218696
ALT                                47445
AST                                  199
Bilirubin                              6
CRP                                  720
CVP                                53989
Creatinine                        148460
Diastolic_BP                        4522
FiO2                              109242
HCO3                              106431
HR                                205363
Haemoglobin                       104816
Mean Arterial Pressure             82324
PCO2                                2590
PEEP_set                           76390
PaCO2                              47840
PaO2                               82681
Plateau_Pressure                   46387
Platelet_count                    105036
Potassium                         148614
RR                                204675
SaO2                              200038
Sodium          

In [36]:
features = df[my_cols]

In [37]:
features.count()

Creatinine                148460
HCO3                      106431
HR                        205363
Haemoglobin               104816
Platelet_count            105036
Potassium                 148614
RR                        204675
Sodium                    147400
Weight_kg                 196484
SaO2                      200038
arterial_bp_systolic      103015
arterial_bp_diastolic     103007
HADM_ID                   218696
CHARTDATE                 218696
DAYS_TO_DISCH             218696
AGE                       218696
GENDER                    218696
Day                       218696
DEATH                     218696
Chronic Kidney Disease    218696
Obesity                   218696
Heart disease             218696
Asthma                    218696
dtype: int64

In [38]:
features = features.dropna()

In [39]:
features.count()

Creatinine                89057
HCO3                      89057
HR                        89057
Haemoglobin               89057
Platelet_count            89057
Potassium                 89057
RR                        89057
Sodium                    89057
Weight_kg                 89057
SaO2                      89057
arterial_bp_systolic      89057
arterial_bp_diastolic     89057
HADM_ID                   89057
CHARTDATE                 89057
DAYS_TO_DISCH             89057
AGE                       89057
GENDER                    89057
Day                       89057
DEATH                     89057
Chronic Kidney Disease    89057
Obesity                   89057
Heart disease             89057
Asthma                    89057
dtype: int64

In [40]:
# number of patients data we still have
features['HADM_ID'].value_counts().count()

21362

In [41]:
# days 
features['Day'].value_counts()

 1      16554
 2      11310
 3       7919
 0       7728
 4       6107
 5       4882
 6       4093
 7       3479
 8       2943
 9       2497
 10      2130
 11      1881
 12      1636
 13      1470
 14      1294
 15      1147
 16      1007
 17       917
 18       805
 19       755
 20       688
 21       634
 22       578
 23       518
 24       476
 25       429
 26       385
 27       344
 28       322
 29       305
 30       275
 31       252
 32       226
 33       211
 34       189
 35       178
 36       159
 37       151
 38       133
 39       125
 40       119
 41       108
 42        98
 43        96
 45        78
 44        77
 46        71
 47        64
 49        63
 48        62
 50        58
 51        51
 54        44
 52        43
 55        41
 53        40
 56        38
 57        35
 58        34
 59        32
 61        30
 60        30
 62        27
 64        27
 63        25
 65        24
 66        21
 72        21
 70        21
 67        20
 73        20
 74   

In [42]:
features['Day'].describe()

count    89057.000000
mean         7.536814
std         10.982553
min         -1.000000
25%          1.000000
50%          4.000000
75%          9.000000
max        177.000000
Name: Day, dtype: float64

In [43]:
# drop days after 2 weeks to match redcap dataset
# but this will make predicting next day discharge really hard because
# 14 day patients won't have the same values as other actually discharged patients

# data = features[features['Day'] < 14]
data = features

In [44]:
data

,Creatinine,HCO3,HR,Haemoglobin,Platelet_count,Potassium,RR,Sodium,Weight_kg,SaO2,arterial_bp_systolic,arterial_bp_diastolic,HADM_ID,CHARTDATE,DAYS_TO_DISCH,AGE,GENDER,Day,DEATH,Chronic Kidney Disease,Obesity,Heart disease,Asthma
0,2.3,17.0,122.0,11.0,376.0,4.2,14.0,143.0,99.0,100.0,192.0,100.0,100001,2117-09-11,6,35,F,0,0,1.0,0.0,0.0,0.0
2,2.0,23.0,117.0,11.2,378.0,3.3,18.0,138.0,99.0,97.0,169.0,102.0,100001,2117-09-13,6,35,F,2,0,1.0,0.0,0.0,0.0
5,1.2,16.0,71.0,7.1,148.0,5.0,13.0,133.0,85.3,99.0,77.0,45.0,100003,2150-04-17,4,60,M,0,0,0.0,0.0,0.0,0.0
6,1.1,15.0,74.0,10.4,162.0,5.0,18.0,133.0,85.3,100.0,109.0,69.0,100003,2150-04-18,4,60,M,1,0,0.0,0.0,0.0,0.0
7,1.0,18.0,81.0,9.6,132.0,4.3,16.0,133.0,85.3,98.0,106.0,87.0,100003,2150-04-19,4,60,M,2,0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218688,1.0,26.0,67.0,9.7,140.0,3.7,13.0,148.0,53.4,93.0,113.0,63.0,199992,2155-02-12,2,86,F,1,0,0.0,0.0,0.0,0.0
218692,0.8,25.0,75.0,11.3,296.0,3.4,27.0,131.0,71.5,98.0,152.0,56.0,199999,2136-04-07,5,89,M,3,0,0.0,0.0,0.0,0.0
218693,0.7,26.0,78.0,11.9,389.0,4.4,29.0,133.0,71.5,96.0,150.0,47.0,199999,2136-04-08,5,89,M,4,0,0.0,0.0,0.0,0.0
218694,0.8,28.0,69.0,12.5,417.0,4.2,26.0,136.0,71.5,89.0,148.0,50.0,199999,2136-04-09,5,89,M,5,0,0.0,0.0,0.0,0.0


TODO Should discharge be only non deaths?
probably

Problem with missing days! e.g. day 0 missing but day 1 present. Ensure data for each HADM_ID is complete. 

TODO will probably be far better to impute data instead.

In [45]:
drop_hadm = []

for hadm in data['HADM_ID'].unique():
#     if len(data[data.HADM_ID == hadm]) - 1 == max(data[data.HADM_ID == hadm]['Day']) \
#             == max(data[data.HADM_ID == hadm]['DAYS_TO_DISCH']):
    if max(data[data.HADM_ID == hadm]['Day']) == max(data[data.HADM_ID == hadm]['DAYS_TO_DISCH']):
        continue
    drop_hadm.append(hadm)

In [46]:
complete_data = data[~data.HADM_ID.isin(drop_hadm)]

In [47]:
complete_data['HADM_ID'].value_counts().count()

2923

In [48]:
complete_data['Day'].value_counts()

1      2275
2      1494
0      1191
3      1080
4       854
5       727
6       664
7       575
8       515
9       473
10      431
11      395
12      355
13      328
14      294
15      278
16      247
17      221
18      199
19      194
20      190
21      176
22      169
23      159
24      140
25      124
26      116
28      102
27      101
29       95
30       81
31       78
32       72
33       70
34       61
35       60
37       54
36       53
38       47
39       42
40       41
41       39
42       34
43       32
45       26
44       26
46       19
49       19
47       19
48       19
50       18
51       17
54       14
55       13
52       12
56       11
53       11
62        9
61        9
60        9
57        9
59        8
58        7
63        7
75        6
64        6
76        6
67        5
74        5
70        5
72        5
68        5
71        5
73        5
66        5
65        5
78        4
106       4
77        4
69        4
80        3
107       3
79        3
113 

In [49]:
complete_data.columns

Index(['Creatinine', 'HCO3', 'HR', 'Haemoglobin', 'Platelet_count', 'Potassium', 'RR', 'Sodium', 'Weight_kg', 'SaO2', 'arterial_bp_systolic', 'arterial_bp_diastolic', 'HADM_ID', 'CHARTDATE', 'DAYS_TO_DISCH', 'AGE', 'GENDER', 'Day', 'DEATH', 'Chronic Kidney Disease', 'Obesity', 'Heart disease', 'Asthma'], dtype='object')

## Remove outliers

delete outliers (e.g. 0 RR, 0 HR, very high heart rate)

calculate zscore of every point, remove rows with any zscores > 4

Note that this number is chosen to ensure RR=0 is not removed. This removes too many patients who died.

In [50]:
zscore = abs(complete_data[numeric] - complete_data[numeric].mean()) / complete_data[numeric].std()
no_outliers = complete_data[(zscore < 4).all(axis=1)]

In [51]:
no_outliers['HADM_ID'].value_counts().count()

2865

In [52]:
no_outliers.to_csv('data_processed/mimic_dataset_incomplete.csv', index=False)